# Test CDR Tools
https://opensourcepack.blogspot.com/p/cdrtools.html

In [ ]:
import subprocess


class ODMedia:
    def __init__(self):
        self.path_root = r"C:\Users\HumphreyDrummond\Downloads\schily-cdrtools-3.02a05\win64"
    
    def version(self):
        result = subprocess.run([self.path_root + r"\cdrecord", "--version"], stdout=subprocess.PIPE)
        return result.stdout.decode('utf-8')

    def drive_type(self):
        result = subprocess.run([self.path_root + r"\cdrecord", "-prcap"], stdout=subprocess.PIPE)
        return result.stdout.decode('utf-8')

    def disk_type(self):
        result = subprocess.run([self.path_root + r"\cdrecord", "-minfo", "-v"], stdout=subprocess.PIPE)
        return result.stdout.decode('utf-8')

media = ODMedia()
print(media.drive_type())
print(media.disk_type())

In [ ]:
import wmi
import os
import time
import ctypes

class ODInfoError(Exception):
    pass

class ODInfo:
    def __init__(self):
        c = wmi.WMI()
        self.num_optical_drives = len(c.Win32_CDROMDrive())
        assert self.num_optical_drives == 1, ODInfoError(
            f'Assume 1 and only 1 drive but have {self.num_optical_drives}')
        self.cdrom = c.Win32_CDROMDrive()[0]
        print('Media loaded', self.media_loaded)
        #print(cdrom)
        #print(cdrom.__dict__)

        for disk in c.Win32_LogicalDisk():
            # Drive Type  from https://blogs.technet.microsoft.com/heyscriptingguy/2014/09/10/inventory-drive-types-by-using-powershell/
            #  2 = "Removable disk"
            #  3="Fixed local disk"
            #  4="Network disk"
            #  5 = "Compact disk
            if int(disk.DriveType) == 5:  # Found the 1 Optical drive
                self.device_id = disk.DeviceID  # Blank
                self.file_system = disk.FileSystem
                self.free_space = disk.FreeSpace
                self.size = disk.Size
        #        print(disk)

    @property
    def media_loaded(self):
        return self.cdrom.MediaLoaded
        
    @property
    def blank_media(self):
        return self.file_system is None
        
    def __str__(self):
        result = self.cdrom.manufacturer + '\n'
        result += f'Blank Media = {self.blank_media}\n'
        if self.media_loaded:
            result += 'Media loaded\n'
            result += f'Device ID = {self.device_id}\n'
            result += f'File system = {self.file_system}\n'
            result += f'Free Space = {self.free_space}\n'
            result += f'Size = {self.size}\n'
        else:
            result += 'No media loaded\n'
        return result

        #for disk in c.Win32_DiskDrive():
        #    print(disk)


        #for media in c.Win32_PhysicalMedia():
        #    if media.tag.find("CDROM") >= 0:
        #        print(media)
        #        if media.Manufacturer != None:
        #            print("Manufacturer:" + media.Manufacturer)

a = ODInfo()
print(a)


In [2]:
import pyIMAPI


In [3]:
disk_types = ["CD", "DVD", "DVDDL", "BluRay"]

for dt in disk_types:
    specific_physical_media_test_filename = dt+"-test.txt"
    t = open(specific_physical_media_test_filename, "w")
    t.write("this "+ dt +" test works")
    t.close()

    o = pyIMAPI.open(dt+"-test.iso", disk_type=dt)
    o.add(specific_physical_media_test_filename)
    o.close()
